In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import pathway as pw
import bokeh.plotting
import panel as pn
from pathway import udf

In [3]:
df = pd.read_csv("dataset.csv")

# Merge date + time to Timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Sort and save simplified streaming CSV
df = df.sort_values("Timestamp").reset_index(drop=True)
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("stream_input.csv", index=False)

In [4]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

In [5]:
# Simulate a real-time stream from static CSV
data = pw.demo.replay_csv("stream_input.csv", schema=ParkingSchema, input_rate=100)

In [6]:
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [7]:
class DynamicPricingModel:
    def __init__(self):
        self.alpha = 1.0
        self.prev_price = 10.0
        self.current_day = None
        self.x_vals = []
        self.deltas = []

    def update_price(self, occupancy, capacity, day):
        if capacity == 0:
            return self.prev_price

        # Reset daily
        if day != self.current_day:
            self.prev_price = 10.0
            self.current_day = day
            self.x_vals.clear()
            self.deltas.clear()

        x = occupancy / capacity
        new_price = self.prev_price + self.alpha * x
        delta = new_price - self.prev_price

        self.x_vals.append(x)
        self.deltas.append(delta)

        # Fit α using linear regression if we have 5+ points
        if len(self.x_vals) >= 5:
            try:
                X = np.array(self.x_vals).reshape(-1, 1)
                Y = np.array(self.deltas)
                X_aug = np.hstack([X, np.ones_like(X)])
                coeffs = np.linalg.lstsq(X_aug, Y, rcond=None)[0]
                fitted_alpha = float(np.clip(coeffs[0], 0, 5))
                self.alpha = fitted_alpha
                print(f"🔁 α updated to {self.alpha:.4f}")
            except Exception as e:
                print("⚠️ α update error:", e)

            self.x_vals.clear()
            self.deltas.clear()

        self.prev_price = new_price
        return new_price


In [8]:
# Instantiate model
model = DynamicPricingModel()

# Windowing: daily tumbling + reduction
import datetime

daily_price_stream = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        day = pw.reducers.any(pw.this.day),
        Occupancy = pw.reducers.avg(pw.this.Occupancy),
        Capacity = pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        PredictedPrice = pw.apply(
            model.update_price,
            pw.this.Occupancy,
            pw.this.Capacity,
            pw.this.day
        )
    )
)


In [10]:
pn.extension()

def plot_price(source):
    fig = bokeh.plotting.figure(
        height=600,
        width=1200,
        title="📈 Dynamic Parking Price (Priceₜ₊₁ = Priceₜ + α·Occ/Cap)",
        x_axis_type="datetime"
    )
    fig.line("t", "PredictedPrice", source=source, line_width=2, color="green")
    fig.circle("t", "PredictedPrice", source=source, size=6, color="red")
    return fig

viz = daily_price_stream.plot(plot_price, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [11]:
pw.run()

Output()